# Testing

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
a = np.array([[1.1,1.2,1.0,0.9,1.1],
              [1.1,1.2,1.0,0.9,1.1],
              [1.1,1.2,1.0,0.9,1.1]]).transpose()
print(a)

In [ ]:
plt.figure(figsize=(3,7))
x = np.array([1,2,3,4,5])
for col in range(len(a[0,:])):
    if col == 0:
        plt.plot(x,a[:,col])
    else:
        pad = len(x) + 1
        x = x + pad
        plt.plot(x,a[:,col])

In [ ]:
for count, i in enumerate(range(len(a[:,0]))):
    print(count)


In [1]:
import re

In [24]:
target_string = "JW0035AAAA0001_Frame.xsglog"

In [25]:
# What I want to get
# name = "JW0035"
# epoch = "AAAA0001"
# channel = "Frame"

In [34]:
a = re.search("[A-Z]{2}[0-9]{4}",target_string).group()


'JW0035'

In [36]:
new_string = target_string.replace(a,"")
new_string

'AAAA0001_Frame.xsglog'